#Práctico 4+5: Introducción a los espacios embebidos de vértices y a las redes neuronales para grafos

# Introducción

En este práctico vamos a trabajar sobre métodos para representar un grafo en un espacio euclideano. 

Como hemos visto en el teórico podemos encontrar formas de representar los vértices de un grafo como vectores. Esto es útil por varios motivos. Por un lado, nos permite aplicar algoritmos tradicionales de aprendizaje automático (Machine Learning - ML) a estos vértices. Por otro, podemos medir distancia en la nueva representación e interpretar que vértices cercanos son similares.

En este práctico vamos a utilizar, en adición a `IGraph`, la biblioteca llamada `StellarGraph` que tiene muchas utilidades para hacer ML en grafos.

In [ ]:
!pip install python-igraph > /dev/null
!pip install cairocffi > /dev/null
!pip install stellargraph > /dev/null

In [ ]:
import igraph as ig
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

# Modules used for node2vec
from stellargraph import StellarGraph
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import datasets
from stellargraph.data import BiasedRandomWalk

from IPython.display import display, HTML

#1) Club de Karate

Vamos a descargar el dataset del club de karate con el que ya hemos trabajado.

In [ ]:
!wget -q "https://raw.githubusercontent.com/prbocca/na101_master/master/homework_04_embeddings/karate.graphml" -O "karate.graphml"

In [ ]:
g_karate = ig.load("karate.graphml")
g_karate.summary()

Como podemos ver, cada nodo cuenta con una facción (`Faction`) que viene dada por el dataset.

In [ ]:
N = np.arange(g_karate.vcount())
g_karate.vcount(), g_karate.ecount()

In [ ]:
g_karate.vs[0].attributes()

Vemos que hay tan solo 2 facciones: `1.0` y `2.0`.

In [ ]:
set([n["Faction"] for n in g_karate.vs])

Visualizamos el grafo según las facciones.

In [ ]:
g_karate.vs["label"] = [s.replace("Actor ", "") for s in g_karate.vs["name"]]

visual_style = dict()
visual_style["layout"] = g_karate.layout()
visual_style["vertex_shape"] = ["rectangle" if name_ in ["Mr Hi", "John A"] else "circle" for name_ in g_karate.vs["name"]]
visual_style["vertex_color"] = ["red" if type_ == 1 else "blue" for type_ in g_karate.vs["Faction"]]
visual_style["vertex_size"] = g_karate.strength()
visual_style["edge_width"] = g_karate.es["weight"]

f1 = g_karate.vs.select(Faction=1)
f2 = g_karate.vs.select(Faction=2)
g_karate.es.select(_between=(f1, f1))["color"] = "pink"
g_karate.es.select(_between=(f2, f2))["color"] = "skyblue"
g_karate.es.select(_between=(f1, f2))["color"] = "yellow"

ig.plot(g_karate, **visual_style)

##1.1) Laplacian Eigenmap

Una de las formas más simples y eficaces de embember los vértices de un grafo en un espacio vectorial se conoce como `Laplacian Eigenmap`. 

Este método es un caso muy sencillo de embebido superficial que sigue el framework `encoder-decoder`. 
* Al ser superficial, el `encoder` es una función de mapeo (tabla): $ENC(u)=\mathbf{Z}[u]$, donde $\mathbf{Z} \in \mathbb{R}^{|V| \times d}$, 
* y el `decoder` es la función de parejas de vectores de vértices definida:
$$ DEC(\mathbf{Z}[u], \mathbf{Z}[v]) = || \mathbf{Z}[u] - \mathbf{Z}[v] ||_2^2.$$

Cuando se entrena el espacio embebido para que la similaridad entre los vectores sea la matriz laplaciana, se puede demostrar (ver teórico) que este método de embebido corresponde a elegir $\mathbf{Z} \sim$ los $d$ vectores propios de $L$ de valor propio más pequeño, exceptuando el valor propio 0.

Aplicaremos este resultado entonces.
Primero, empezamos por construir el laplaciano del grafo:

$$ L = D - A.$$

Esto es bastante sencillo de hacer con `igraph`.

In [ ]:
L = np.array(g_karate.laplacian())

In [ ]:
#podemos imprimir la matriz, o verla como una imagen
display(L)

plt.imshow(L)
plt.show()

Luego, tenemos que calcular los valores y vectores propios del laplaciano. Esto también es fácil de hacer usando `numpy`.

In [ ]:
eig_val, eig_vec = np.linalg.eig(L)
print(eig_val)

Vamos a necesitar los vectores propios ordenados de menos a mayor.

In [ ]:
idx = eig_val.argsort()
eig_val = eig_val[idx]
eig_vec = eig_vec[:, idx]

print("Los valores propios son:\n", eig_val)
print("Los vectores propios estan en una matriz NxN:\n", eig_vec.shape)

En el espacio de vectores propios, a cada vértice le corresponde un vector de tamaño $N$ (donde $N$ son la cantidad de nodos en el grafo, en este caso 34).

Luego, para embeber los vértices en un espacio de dimensión $d$ vamos a quedarnos con las $d$ coordenadas de cada nodo asociadas a los valores propios más chicos (sin contar el más chico de todos, ese lo ignoramos).

Veamos como ejemplo sencillo $d=2$.

In [ ]:
d = 2
encoder_laplacian_eigenmap = eig_vec[:, 1: 1 + d] # El eje 0 es la cantidad de nodos y el eje 1 es la dimensión

node_targets_laplacian_eigenmap = pd.Categorical(g_karate.vs["Faction"]).astype("category")

print("El embebido superficial tiene dimensiones ", encoder_laplacian_eigenmap.shape)

El espacio embebido $\mathbf{Z} \in \mathbb{R}^{34 \times 2}$ de dimensión $d=2$ para los 34 vértices (ordenados según el laplaciano) es:

In [ ]:
encoder_laplacian_eigenmap

Una vez que tenemos la representación de cada nodo, podemos graficarlos. Esto es sencillo si usamos dimensión 2 (si hubiesemos elegido $d=5$ sería más difícil).

Como podemos ver, esta representación hace bastante fácil separar linealmente los nodos de ambas facciones (lo cuál es una muy buena noticia par un algoritmo de inteligencia aritificial!)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
for n in N:
  faction = node_targets_laplacian_eigenmap[n] 
  color = "red" if faction == 1 else "blue"
  x,y = encoder_laplacian_eigenmap[n, :]
  ax.scatter(x, y, c=color, label=faction)

ax.set_xlabel("1st non zero eignvector")
ax.set_ylabel("2nd non zero eignvector")

handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())
plt.show()

##1.2) Node2Vec

Otro algoritmo bastante popular y bastante más "poderoso" es `node2Vec`.

La idea de este algoritmo es muy ingeniosa: dado que el algoritmo `word2vec` es muy bueno en representar palabras como vectores a partir de oraciones, porque no encontrar la forma de transformar vértices en palabras y reutilizar el mismo algoritmo? Y como hacemos para poder hacer esa transformación?

Para entrenar un algoritmo de `node2vec` lo que se necesitan son oraciones hechas a partir de palabras.

Para poder hacer esto mismo en el grafo, la idea es generar "caminos": que cada nodo sea interpretado como una palabra y que el camino entero sea una oración.



Como habíamos mencionado antes, vamos a utilizar la bliblioteca `StellarGraph` y para esto necesitamos cargar el grafo desde `networkx`.

In [ ]:
g_karate.write_graphml("g_karate.graphml") #guardo el grafo igraph con los atributos generados, para luego leerlo desde networkx

In [ ]:
G = nx.read_graphml("g_karate.graphml")
print(nx.info(G))

In [ ]:
G_S = StellarGraph.from_networkx(G)

In [ ]:
print(G_S.info()[:1000])

Una de las ideas que hacen que `node2vec` funcione tan bien es la forma se generan los caminos en el grafo.

La forma "fácil" sería generar muchas caminatas aleatorias a partir de cada nodo del grafo y usar eso. 

En lugar de eso, el algoritmo asigna distintos pesos a cada arista a la hora de generar una muestra aleatoria, haciendo el método más efectivo.

En `StellarGraph` el método `BiasedRandomWalk` genera caminatas aleatorias de este estilo.

In [ ]:
rw = BiasedRandomWalk(G_S)

walks = rw.run(
    nodes=list(G_S.nodes()),  # root nodes
    length=80,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
)
print("Number of random walks: {}".format(len(walks)))

Así es como se ve una caminata aleatoria: una sequencia de nodos de largo 80.

In [ ]:
walks[0][:10]

Ahora que tenemos nuestras oraciones de nodos, lo único que tenemos que hacer es importar una implementación de `Word2Vec` y entrenar!

Nuevamente usaremos como dimensión $d=2$.

In [ ]:
from gensim.models import Word2Vec

d = 2

str_walks = [[str(n) for n in walk] for walk in walks]
model = Word2Vec(str_walks, size=d, window=5, min_count=0, sg=1, workers=2, iter=1)
encoder_node2vec = np.vstack([model.wv[f"n{i}"] for i in N])

El espacio embebido $\mathbf{Z} \in \mathbb{R}^{34 \times 2}$ de dimensión $d=2$ para los 34 vértices es:

In [ ]:
encoder_node2vec

Una vez más, como elegimos dimension $d=2$, podemos graficar el resultado del algoritmo en el plano.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
for n in N:
  faction = int(g_karate.vs[n]["Faction"])
  color = "red" if faction == 1 else "blue"
  x,y = encoder_node2vec[n, :]
  ax.scatter(x, y, c=color, label=faction)

ax.set_xlabel("1st non zero eignvector")
ax.set_ylabel("2nd non zero eignvector")

handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

##1.3) Entrenando un clasificador de facciones

Hasta ahora hemos definido dos algoritmos para transformar nodos en vectores de dimensión 2. Es hora de ponerlos en uso.

Para esto, vamos a entrenar un clasificador bastante simple que decida a que facción pertenece cada nodo.

Como "features" vamos a usar la representación obtenida por cada uno de los algoritmos. Vamos a usar algunos de los nodos como conjunto de `train` y otros como conjunto de `test`.



In [ ]:
df_laplacian = pd.DataFrame(encoder_laplacian_eigenmap, columns=["f1", "f2"])
df_node2vec = pd.DataFrame(encoder_node2vec, columns=["f1", "f2"])

NAMES = ["Laplacian Eigenmap", "Node2Vec"]
dfs = [df_laplacian, df_node2vec]
for df in dfs:
  df["target"] = [True if x == 1 else False for x in g_karate.vs["Faction"]]

In [ ]:
display(dfs[0].head())
display(dfs[1].head())

 Elegimos el conjunto de train con 17 vértices al azar.

In [ ]:
r = np.random.RandomState(1234)
train_index = r.choice(range(34), size=17, replace=False)

display(train_index)

Lo que vamos a hacer aquí es para cada dataset, separar el conjunto de train y test para luego entrenar una regresión.

Como sabemos, la regresión no puede aceptar grafos como entrada, pero no tiene problema en aceptar pares de nodos.

In [ ]:
models = []
for i, df in enumerate(dfs):
  train_df = df[df.index.isin(train_index)]
  test_df = df[~df.index.isin(train_index)]
  y_train = train_df.pop("target")
  y_test = test_df.pop("target")

  log_reg = LogisticRegression(random_state=r)
  log_reg.fit(train_df, y_train)
  predicted = log_reg.predict(test_df)

  acc = accuracy_score(y_test, predicted)
  models.append(log_reg)

  print(f"Model {NAMES[i]: <20} has an accuracy of {acc:0.4f}")

Podemos ir incluso un paso más allá y dibujar las regiones que encuentra el clasificador para ambos algoritmos. Aquí es más fácil de ver que `node2vec` es incluso un poco mejor.

In [ ]:
h = 0.02
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
for i in range(2):
  ax = axs[i]
  df = dfs[i]
  model = models[i]

  x_min, x_max = df["f1"].min(), df["f1"].max()
  y_min, y_max = df["f2"].min(), df["f2"].max()
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                      np.arange(y_min, y_max, h))

  Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
  Z = Z.reshape(xx.shape)

  ax.contourf(xx, yy, Z, cmap=plt.cm.coolwarm, alpha=0.7)

  for _, row in df.iterrows():
    x, y = row["f1"], row["f2"]
    color = "red" if row["target"] else "blue"
    ax.scatter(x, y, marker='.', c=color)
  
  ax.set_title(NAMES[i])
  ax.set_xlabel("f1")
  ax.set_ylabel("f2")

##1.4)  Entrenando una Graph Neural Network (GNN) con `StellarGraph`

Esta parte del práctico está basada en el siguiente [blog post](https://stellargraph.readthedocs.io/en/stable/demos/node-classification/gcn-node-classification.html)

In [ ]:
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN

from tensorflow.keras import layers, optimizers, losses, metrics, Model
from sklearn import preprocessing, model_selection

Recordamos que tenemos el grafo en formato `StellarGraph`. 
Como vamos a ver, el grafo todavía no tiene los atributos necesarios. Estos los vamos a agregar a continuación.

In [ ]:
print(G_S.info())

Lamentablemente el grafo no tiene los atributos en los vértices que necesitamos. Por tanto, mostramos otra forma de cargar el grafo.

Para cargar el grafo en `StellarGraph` pasaremos de `igraph` a `pandas` y luego a `StellarGraph`. Para esto primero creamos una lista de aristas (no dirigidas y las guardamos en un dataframe). Y luego creamos un dataframe de atributos de vértices.

In [ ]:
edges = pd.DataFrame([[e.source, e.target] for e in g_karate.es],
                     columns = ["source", "target"])
display(edges)

node_features = pd.DataFrame({"class": [0 if type_ == 1 else 1 for type_ in g_karate.vs["Faction"]]})
display(node_features.head())

Creamos el grafo con atributos.

In [ ]:
G_S = sg.StellarGraph(node_features, edges) #agrego comunidad

print(G_S.info()) #ahora tiene el atributo 'class'

Ahora tenemos que elegir que nodos vamos a usar para entrenar y cuales para testing. Esto podríamos hacerlo con la máscara definida anteriormente, pero optamos por sortearlos nuevamente con fines ilustrativos. 

Atención: Esto **no significa que vayamos a usar un subgrafo para entrenar**. Simplemente que a la hora de calcular la función objetivo vamos a tener en cuenta solo alguno de los vértices (pero usaremos todo el grafo para construir la red de mensajes).

En particular, vamos a elegir solo 2 vértices de cada clase.

In [ ]:
train = node_features.groupby("class").sample(2, random_state=42).copy()
train

El resto de los vértices serán para testing.

In [ ]:
test = node_features[~node_features.index.isin(train.index)].copy()
display(test.head())

print(test.shape) # 26 + 8 = 34

Para usar StellarGraph necesitamos transformar la categoría clase en una lista de atributos binarios donde cada columna indica si el nodo es de dicha clase o no. Esto se conoce como `One Hot Encoding`

In [ ]:
train_dummy = pd.get_dummies(train["class"])
display(train_dummy.head())

test_dummy = pd.get_dummies(test["class"])
display(test_dummy.head())

Los próximos pasos son necesarios para que StellarGraph pueda trabajar con los datos que les pasamos.

In [ ]:
generator = FullBatchNodeGenerator(G_S, method="gcn") # 

In [ ]:
train_gen = generator.flow(
    train_dummy.index, # Estos son los indices usados para el set de train
    train_dummy.values, # Estos es una matrix de numpy con los valores correspondientes a las clases
    use_ilocs=False # Esto se usa para indicar que los indices son "nombres" y no posisciones
)

In [ ]:
gcn = GCN(layer_sizes=[4, 4, 2], activations=["tanh", "tanh", "tanh"], generator=generator, dropout=0)

In [ ]:
x_inp, x_out = gcn.in_out_tensors()

In [ ]:
predictions = layers.Dense(units=train_dummy.shape[1], activation="softmax")(x_out)

In [ ]:
model = Model(inputs=x_inp, outputs=predictions)
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.categorical_crossentropy,
    metrics=["acc"],
)

In [ ]:
history = model.fit(
    train_gen,
    epochs=200,
    verbose=2,
    shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
)

In [ ]:
sg.utils.plot_history(history)

Finalmente, podemos revisar el resultado de la clasificación en el set de test.

In [ ]:
test_gen = generator.flow(test_dummy.index, test_dummy.values)

In [ ]:
out = model.predict(test_gen)

print(out.shape)

In [ ]:
print("Las comunidades reales: \n", test['class'].to_numpy()) 

print("Las comunidades predichas: \n", out[0].argmax(axis=1)) # Calcular la clasificación de cada nodo

In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

Y en todo el conjunto de datos, para compararnos con las soluciones anteriores.

In [ ]:
all_dummy = pd.get_dummies(node_features["class"])
display(all_dummy.head())

all_gen = generator.flow(all_dummy.index, all_dummy.values)

out = model.predict(all_gen)
print(out.shape)

print("Las comunidades reales: \n", node_features['class'].to_numpy()) 
print("Las comunidades predichas: \n", out[0].argmax(axis=1)) # Calcular la clasificación de cada nodo

#2) CORA: grafo grande de publicaciones científicas

CORA tiene $2708$ vértices que representan publicaciones. 

Cada publicación pertenece a una de $7$ clases posibles.

Se conocen las citas entre publicaciones, en un grafo con $5429$ aristas.

La biblioteca `stellarGraph` incluye este grafo dentro de sus ejemplos. Lo cargamos.

In [ ]:
def jaccard_weights(graph, _subjects, edges):
    sources = graph.node_features(edges.source)
    targets = graph.node_features(edges.target)

    intersection = np.logical_and(sources, targets)
    union = np.logical_or(sources, targets)

    return intersection.sum(axis=1) / union.sum(axis=1)

dataset = datasets.Cora()
display(HTML(dataset.description))
G_S, subjects = dataset.load(
    largest_connected_component_only=True,
    edge_weights=jaccard_weights,
    str_node_ids=True,  # Word2Vec requires strings, not ints
)

N = np.arange(G_S.number_of_nodes())
print(G_S.info())

## 4.1) Laplacian Eigenmap

Calculemos el laplaciano, para eso una opción es convertir el grafo a `networkx`y usar la función `laplacian_matrix()`.

In [ ]:
G = G_S.to_networkx()
print(nx.info(G))

In [ ]:
### START CODE HERE
### END CODE HERE

#podemos imprimir la matriz, o verla como una imagen
display(L)

plt.imshow(L)
plt.show()

In [ ]:
# Tip: ver ejemplo anterior
### START CODE HERE
### END CODE HERE

print("Los valores propios son:\n", eig_val)
print("Los vectores propios estan en una matriz NxN:\n", eig_vec.shape)

Calcular el espacio embebido Laplacian Eigenmap de dimensión $128$.

In [ ]:
d = 128

# Tip: ver ejemplo anterior
### START CODE HERE
### END CODE HERE

node_targets_laplacian_eigenmap = subjects.astype("category")

print("El embebido superficial tiene dimensiones ", encoder_laplacian_eigenmap.shape)

Vamos a visualizar en 2D las categorias y los vectores embebidos.

In [ ]:
# Apply t-SNE transformation on node embeddings
tsne = TSNE(n_components=2, random_state=42)
encoder_laplacian_eigenmap_2d = tsne.fit_transform(encoder_laplacian_eigenmap)

# draw the points
alpha = 0.7

plt.figure(figsize=(10, 8))
plt.scatter(
    encoder_laplacian_eigenmap_2d[:, 0],
    encoder_laplacian_eigenmap_2d[:, 1],
    c=node_targets_laplacian_eigenmap.cat.codes,
    cmap="jet",
    alpha=0.7,
)
plt.show()

## 4.2) Node2vec

Generamos caminantes con los siguientes parámetros.

In [ ]:
rw = BiasedRandomWalk(G_S)

walks = rw.run(
    nodes=G_S.nodes(),  # root nodes
    length=100,  # maximum length of a random walk
    n=10,  # number of random walks per root node
    p=0.5,  # Defines (unormalised) probability, 1/p, of returning to source node
    q=2.0,  # Defines (unormalised) probability, 1/q, for moving away from source node
    weighted=True,  # for weighted random walks
    seed=42,  # random seed fixed for reproducibility
)
print("Number of random walks: {}".format(len(walks)))

Entrenar `node2vec`.

In [ ]:
from gensim.models import Word2Vec

d = 128

# Tip: ver ejemplo anterior
### START CODE HERE
### END CODE HERE


In [ ]:
# Retrieve node embeddings and corresponding subjects
node_ids = model.wv.index2word  # list of node IDs
encoder_node2vec = (
    model.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
# the gensim ordering may not match the StellarGraph one, so rearrange
node_targets_node2vec = subjects.loc[node_ids].astype("category")

display(encoder_node2vec.shape)

Vamos a visualizar en 2D las categorias y los vectores embebidos.

In [ ]:

# Tip: ver ejemplo anterior
### START CODE HERE
### END CODE HERE


## 4.3) Clasificación de vértices

Vamos a predecir la categoría de las publicaciones usando una regresión logística.

In [ ]:
df_laplacian = pd.DataFrame(encoder_laplacian_eigenmap)
df_laplacian['target']= pd.Categorical(node_targets_laplacian_eigenmap)

df_node2vec = pd.DataFrame(encoder_node2vec)
df_node2vec['target']= pd.Categorical(node_targets_node2vec)

NAMES = ["Laplacian Eigenmap", "Node2Vec"]
dfs = [df_laplacian, df_node2vec]

 Elegimos el conjunto de train con 1863 vértices al azar.

In [ ]:
r = np.random.RandomState(5434)

# Tip: ver ejemplo anterior
### START CODE HERE
### END CODE HERE

display(train_index)

Lo que vamos a hacer aquí es para cada dataset, separar el conjunto de train y test para luego entrenar una regresión.

Como sabemos, la regresión no puede aceptar grafos como entrada, pero no tiene problema en aceptar pares de nodos.

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

models = []
for i, df in enumerate(dfs):
  train_df = df[df.index.isin(train_index)]
  test_df = df[~df.index.isin(train_index)]
  y_train = train_df.pop("target")
  y_test = test_df.pop("target")

  log_reg = LogisticRegressionCV(Cs=10,
                                cv=10,
                                tol=0.001,
                                max_iter=1000,
                                scoring="accuracy",
                                verbose=False,
                                multi_class="ovr",
                                random_state=r)
  log_reg.fit(train_df, y_train)
  predicted = log_reg.predict(test_df)

  acc = accuracy_score(y_test, predicted)
  models.append(log_reg)

  print(f"Model {NAMES[i]: <20} has an accuracy of {acc:0.4f}")

El resultado debe ser:

* Model Laplacian Eigenmap   has an accuracy of 0.7733
* Model Node2Vec             has an accuracy of 0.7958 0.8103